# Exploratory Data Analysis

In this EDA, I will be examining various data sources from [opendata.vancouver.ca](https://opendata.vancouver.ca/pages/home/).

## Imports

In [9]:
import numpy as np
import pandas as pd
import re
import requests
import os

from matplotlib import pyplot as plt
from pyspark.sql import SparkSession

In [25]:
from pyspark.sql.functions import coalesce

In [26]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import array, col, count, mean, sum, udf, when
from pyspark.sql.types import DoubleType, IntegerType, StringType, Row
from pyspark.sql.functions import sum, col, udf
from pyspark.sql.types import *

## Reading in the main dataset

In [27]:
spark = SparkSession.builder.appName('Ops').getOrCreate()

In [28]:
# allows for cleaner output
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [29]:
licences_df = (spark.read
          .option('header','true')
          .option('inferSchema','true')
          .option('sep', ';')
          .csv('data/business-licences.csv'))

In [30]:
licences_df.count()

443158

> There are 443 158 observations in this dataset. 

In [31]:
licences_df.printSchema()

root
 |-- FOLDERYEAR: integer (nullable = true)
 |-- LicenceRSN: integer (nullable = true)
 |-- LicenceNumber: string (nullable = true)
 |-- LicenceRevisionNumber: integer (nullable = true)
 |-- BusinessName: string (nullable = true)
 |-- BusinessTradeName: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- IssuedDate: timestamp (nullable = true)
 |-- ExpiredDate: timestamp (nullable = true)
 |-- BusinessType: string (nullable = true)
 |-- BusinessSubType: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- House: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Province: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- PostalCode: string (nullable = true)
 |-- LocalArea: string (nullable = true)
 |-- NumberofEmployees: double (nullable = true)
 |-- FeePaid: integer (nullable = true)
 |-- ExtractDate: timestamp (nullable = true)
 |-- Geom: st

In [64]:
licences_df.limit(10)

FOLDERYEAR,LicenceRSN,LicenceNumber,LicenceRevisionNumber,BusinessName,BusinessTradeName,Status,IssuedDate,ExpiredDate,BusinessType,BusinessSubType,Unit,UnitType,House,Street,City,Province,Country,PostalCode,LocalArea,NumberofEmployees,FeePaid,ExtractDate,Geom
15,2380055,15-150315,0,Brandi Nicole Eue...,Spa Haven Boutiqu...,Gone Out of Business,null,null,Therapeutic Touch...,null,null,null,null,null,Vancouver,BC,CA,null,Kensington-Cedar ...,1.0,null,2019-07-21 13:49:17,null
15,2380056,15-150316,0,Kelly Ashley Bake...,null,Pending,null,null,Therapeutic Touch...,null,null,null,null,null,Vancouver,BC,CA,null,Kitsilano,1.0,null,2019-07-21 13:49:17,null
15,2380058,15-150318,0,Heart To Mind Cra...,Heart To Mind,Issued,2014-12-20 10:39:10,2015-12-31 00:00:00,Therapeutic Touch...,null,null,null,null,null,Vancouver,BC,CA,null,Kerrisdale,1.0,133,2019-07-21 13:49:17,null
15,2380062,15-150322,0,(Donna Sam),null,Issued,2014-12-02 15:12:32,2015-12-31 00:00:00,Therapeutic Touch...,null,null,null,null,null,Vancouver,BC,CA,null,South Cambie,0.0,155,2019-07-21 13:49:17,null
15,2380065,15-150325,0,James Joseph Thom...,James Thompson RST,Gone Out of Business,null,null,Therapeutic Touch...,null,null,null,null,null,Vancouver,BC,CA,null,Grandview-Woodland,0.0,null,2019-07-21 13:49:17,null
15,2380066,15-150326,0,(Bunchu Praichit),Bunchu Praichit T...,Issued,2014-12-09 21:26:32,2015-12-31 00:00:00,Therapeutic Touch...,null,null,null,null,null,Vancouver,BC,CA,null,Kensington-Cedar ...,0.0,133,2019-07-21 13:49:17,null
15,2380078,15-150338,0,Tourland Travel Ltd,null,Issued,2014-11-24 10:30:17,2015-12-31 00:00:00,Travel Agent,null,7,Unit,900,W GEORGIA ST,Vancouver,BC,CA,V6C 2W6,Downtown,2.0,133,2019-07-21 13:49:17,"""{""""type"""": """"Poi..."
15,2380079,15-150339,0,Jetway Travel Inc,null,Issued,2015-02-27 14:16:59,2015-12-31 00:00:00,Travel Agent,null,null,null,1796,RENFREW ST,Vancouver,BC,CA,V5M 3H8,Hastings-Sunrise,2.0,173,2019-07-21 13:49:17,"""{""""type"""": """"Poi..."
15,2380080,15-150340,0,Skyline Connectio...,null,Issued,2014-12-29 11:34:29,2015-12-31 00:00:00,Travel Agent,null,null,null,5318,VICTORIA DRIVE,Vancouver,BC,CA,V5P 3V7,Kensington-Cedar ...,2.0,133,2019-07-21 13:49:17,"""{""""type"""": """"Poi..."
15,2380082,15-150342,0,The Flight Shops Inc,Flight Centre,Issued,2014-12-03 09:37:03,2015-12-31 00:00:00,Travel Agent,null,null,null,1232,DAVIE ST,Vancouver,BC,CA,V6E 1N3,West End,4.0,133,2019-07-21 13:49:17,"""{""""type"""": """"Poi..."


In [175]:
licences_df.groupBy('FOLDERYEAR').count()

FOLDERYEAR,count
null,1
13,60915
16,61394
20,62366
94,1
96,11
19,70771
15,60938
14,60581
18,66180


## Initial Wrangling 

In [32]:
business_types = licences_df.select('BusinessType').distinct().collect()
business_list = [business_types[i].BusinessType for i in range(len(business_types))]

r = re.compile(".*[Ff]ood.*|.*[Rr]estaurant.*|.*[Ll]iquor.*")
newlist = list(filter(r.match, business_list)) 
print(newlist)

['Manufacturer - Food with Anc. Retail', 'Ltd Service Food Establishment', 'Temp Liquor Licence Amendment', 'Liquor Delivery Services', 'Liquor Establishment Standard', 'Retail Dealer - Food', 'Restaurant Class 2', 'Food Processing', 'Liquor Establishment Extended', 'Wholesale Dealer - Food with Anc. Retail', 'Restaurant Class 1', 'Liquor License Application', 'Warehouse Operator - Food', 'Manufacturer - Food', 'Liquor Retail Store', 'Wholesale Dealer - Food']


> These are the list of business types that have the word food, restaurant, or liquor.

In [90]:
rest_1 = (licences_df.BusinessType == 'Restaurant Class 1')
rest_2 = (licences_df.BusinessType == 'Restaurant Class 2')
rest_3 = (licences_df.BusinessType == 'Ltd Service Food Establishment')
rest_4 = (licences_df.BusinessType == 'Temp Liquor Licence Amendment') & (licences_df.BusinessSubType == 'Area Extension')
rest_5 = (licences_df.BusinessType == 'Liquor Establishment Standard') & (licences_df.BusinessSubType == 'Class 1  0-65 Seats')
rest_6 = (licences_df.BusinessType == 'Liquor Establishment Extended') & (licences_df.BusinessSubType == 'Class 1  0-65 Seats')


licences_rest_df = licences_df.filter(rest_1 | rest_2 | rest_3 | rest_4 | rest_5 | rest_6)

In [91]:
licences_rest_df.groupBy("Status").count()

Status,count
Cancelled,515
Gone Out of Business,2505
Issued,21285
Inactive,541
Pending,1393


In [92]:
print(licences_rest_df.filter('BusinessTradeName is null').select('BusinessName').distinct().count())

933


> There are 933 businesses with missing name values so I will replace these entries with their business names. 

In [93]:
licences_rest_df = licences_rest_df.withColumn('BusinessTradeName', coalesce('BusinessTradeName', 'BusinessName'))

In [94]:
licences_rest_df = licences_rest_df.dropna(subset = ['BusinessName'])

In [95]:
licences_rest_df.count()

26214

In [66]:
from pyspark.sql.functions import regexp_extract, col

# Search for names using regex

def search_restaurant(search_term, n):
    rows = licences_rest_df.BusinessTradeName.rlike('.*{}.*'.format(search_term))
    return licences_rest_df.filter(rows).limit(n)

search_restaurant('Spot', 5)

FOLDERYEAR,LicenceRSN,LicenceNumber,LicenceRevisionNumber,BusinessName,BusinessTradeName,Status,IssuedDate,ExpiredDate,BusinessType,BusinessSubType,Unit,UnitType,House,Street,City,Province,Country,PostalCode,LocalArea,NumberofEmployees,FeePaid,ExtractDate,Geom
16,2603928,16-145091,0,White Spot Limited,White Spot Restau...,Issued,2015-12-24 11:57:18,2016-12-31 00:00:00,Restaurant Class 1,With Liquor Service,null,null,650,W 41st Av,Vancouver,BC,CA,V5Z2M9,Oakridge,0.0,2015,2019-07-21 13:49:20,"""{""""type"""": """"Poi..."
16,2604217,16-145379,0,Top Edge Holdings...,White Spot Restau...,Issued,2015-12-10 13:21:40,2016-12-31 00:00:00,Restaurant Class 1,With Liquor Service,null,null,5367,WEST BOULEVARD,Vancouver,BC,CA,V6M 3W4,Arbutus-Ridge,40.0,1806,2019-07-21 13:49:20,"""{""""type"""": """"Poi..."
16,2604226,16-145388,0,Pacific Internati...,White Spot Restau...,Issued,2015-11-12 12:33:43,2016-12-31 00:00:00,Restaurant Class 1,No Liquor Service,null,null,1881,Cornwall Av,Vancouver,BC,CA,V6J 1C6,Kitsilano,10.0,714,2019-07-21 13:49:20,"""{""""type"""": """"Poi..."
16,2604241,16-145403,0,No 109 Seabright ...,White Spot Restau...,Issued,2015-11-05 13:24:16,2016-12-31 00:00:00,Restaurant Class 1,With Liquor Service,null,null,1476,Kingsway,Vancouver,BC,CA,V5N 2R5,Kensington-Cedar ...,49.0,1688,2019-07-21 13:49:20,"""{""""type"""": """"Poi..."
16,2604304,16-145466,0,White Spot Limite...,White Spot Restau...,Issued,2015-12-24 11:59:33,2016-12-31 00:00:00,Restaurant Class 1,With Liquor Service,null,null,1126,SE Marine Dr,Vancouver,BC,CA,V5X 2V7,Sunset,0.0,1588,2019-07-21 13:49:20,null


> Use an external dataset such as yelp or zomato to get restuarant cuisine, price range, and other features. 

In [96]:
# use this table to populate the features from yelp

rest_distinct_df = licences_rest_df.select(['BusinessTradeName', 'Geom', 'PostalCode', 'House', 'Street']).distinct()

In [97]:
rest_distinct_df.count()

6416

> There are 6416 distinct restaurants locations.

In [98]:
rest_distinct_df.filter('Geom is null').filter('PostalCode is null').filter('House is null').filter('Street is null')


BusinessTradeName,Geom,PostalCode,House,Street
Siga-Siga's Chine...,null,null,null,null
Siga-Siga's Filip...,null,null,null,null
Panz Veggie,null,null,null,null
Fiery Pizza,null,null,null,null


> 4 of these restaurants do not have any information on the location. Since this is a very small proportion of the data, I will remove them.

In [99]:
rest_distinct_df = rest_distinct_df.dropna(how = 'all', subset = ['Geom', 'PostalCode', 'House', 'Street'])
rest_distinct_df.count()

6412

## YELP API

https://www.yelp.ca/developers/documentation/v3/get_started   
https://github.com/Yelp/yelp-fusion#code-samples


In [144]:
test = rest_distinct_df.take(3)

In [147]:
Point = 'point' 
coordinates = 'coordinates' 

In [149]:
eval(eval(test[1].Geom))#[coordinates]

{type: 'point', 'coordinates': [-123.122503805717, 49.2630246879708]}

In [573]:
api_key = os.getenv('YELP_API_KEY')

# defines the values inside Geom 
Point = 'point' 
coordinates = 'coordinates' 


data = dict(names = [],
            house = [], 
            street = [],
            postalcode = [],
            geom = [],
            category = [],
            rating = [],
            price = [])

for i in range(len(test)):

    # append to lists
    data['name'].append(test[i].BusinessTradeName)
    data['house'].append(test[i].House)
    data['street'].append(test[i].Street)
    data['postalcode'].append(test[i].PostalCode)
    data['geom'].append(test[i].Geom)

    # api parameters
    name = test[i].BusinessTradeName
    loc = test[i].House + ' ' + test[i].Street + ' ' + test[i].PostalCode
    coord = eval(eval(test[i].Geom))[coordinates]
    long = coord[0]
    lat = coord[1]

    headers = {'Authorization': 'Bearer %s' % api_key}
    params = {'apikey': api_key,
              'term': name,
              'location': loc,
              'longitude': long,
              'latitude': lat,
              'limit': 1}
    
    request = requests.get('https://api.yelp.com/v3/businesses/search', 
                           headers = headers, params = params)
    
    result = request.json()

    # category
    if result['total'] == 0:
        data['category'].append(None)
        data['rating'].append(None)
        data['price'].append(None)
        data['review_count'].append(None)
        
    else:
        data['category'].append(result['businesses'][0]['categories'])
        data['rating'].append(result['businesses'][0]['rating'])
        data['review_count'].append(result['businesses'][0]['review_count'])
        data['price'].append(result['businesses'][0]['price'])


In [574]:
pd.DataFrame(data)

,BusinessTradeName,Geom,House,Street,category,rating,price
0,Morris J. Wosk Centre for Dialogue,"""{""""type"""": """"Point"""", """"coordinates"""": [-123....",580,W Hastings St,None,NaN,None
1,Banana Leaf Malaysian Cuisine,"""{""""type"""": """"Point"""", """"coordinates"""": [-123....",820,W Broadway,"[{'alias': 'malaysian', 'title': 'Malaysian'},...",3.5,$$
2,Top of Vancouver,"""{""""type"""": """"Point"""", """"coordinates"""": [-123....",555,W HASTINGS ST,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,$$$$


> After removing the restaurants with no location, we have 6349 distinct restaurant locations.

In [134]:
params = {'apikey': api_key,
          'location': '1126 SE Marine Dr V5X 2V7',
          'longitude': -123.156010,
          'latitude': 44.237763,
          'limit': 5,
          'sort_by': 'distance'}
a = requests.get('https://api.yelp.com/v3/businesses/search', headers = headers, params = params)

In [135]:
a.json()['businesses'][0]['review_count']

{'businesses': [{'id': 'Vlfxpl1n5xEher4nn-1F6Q',
   'alias': 'junkyard-extreme-burgers-and-brats-junction-city',
   'name': 'Junkyard Extreme Burgers & Brats',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/Cnwj6-rwory5mHj7lCfnqg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/junkyard-extreme-burgers-and-brats-junction-city?adjust_creative=5sgRUqt0gRSL0aY24fgXOg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=5sgRUqt0gRSL0aY24fgXOg',
   'review_count': 153,
   'categories': [{'alias': 'hotdog', 'title': 'Hot Dogs'},
    {'alias': 'burgers', 'title': 'Burgers'},
    {'alias': 'hotdogs', 'title': 'Fast Food'}],
   'rating': 4.5,
   'coordinates': {'latitude': 44.2532046, 'longitude': -123.19512},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '95410 Hwy 99 E',
    'address2': None,
    'address3': None,
    'city': 'Junction City',
    'zip_code': '97448',
    'country': 'US',
    'state': 'OR',
    'display_addres

## ZOMATO API

https://developers.zomato.com/api  
https://github.com/RapidSoftwareSolutions/Marketplace-Zomato-Package   
api key: 99beac6a17fef3d9816ec61eb532bab0

In [169]:
import os

print(os.getenv('ZOMATO_API_KEY'))

None


In [141]:
# categories
params = {'apikey': '99beac6a17fef3d9816ec61eb532bab0',
          'city_id': None}
categories = requests.get('https://developers.zomato.com/api/v2.1/categories', params = params).json()


In [143]:
# cuisines
params = {'apikey': '99beac6a17fef3d9816ec61eb532bab0',
          'city_id': 256}
cuisines = requests.get('https://developers.zomato.com/api/v2.1/cuisines', params = params).json()


In [145]:
# types
params = {'apikey': '99beac6a17fef3d9816ec61eb532bab0',
          'city_id': 256}
types = requests.get('https://developers.zomato.com/api/v2.1/establishments', params = params).json()

In [166]:
# restaurants
params = {'apikey': '99beac6a17fef3d9816ec61eb532bab0',
          'entity_id': 256,
          'entity_type': 'city',
          'start': 80,
          'count': 100}
restaurants = requests.get('https://developers.zomato.com/api/v2.1/search', params = params).json()

In [170]:
requests.get('https://api.yelp.com/v3/businesses/search')

<Response [400]>